# Harmonize metafiles

In [3]:
library(tidyverse)
library(GEOquery)

# norm_vs_HGSC

norm_vs_HGSC = ["GSE6008_1", "GSE14407_1", "GSE26712_1",
        "GSE40595_1", "GSE36668_1", "GSE69428_1"]

! Using only control samples and HGSC samples

## Load metadata

In [4]:
options(timeout = max(300, getOption("timeout")))
options(download.file.method.GEOquery = "wget")

metadata_files <- list()
samples_IDs <- NULL

for(dataset in c(
    "GSE6008", "GSE14407", "GSE26712",  "GSE40595", "GSE36668", "GSE69428"
    )){

    # check if dir exists
    if(!dir.exists(paste0("before/", dataset))){
        dir.create(paste0("before/", dataset))
    }

    gset <- getGEO(dataset, GSEMatrix=TRUE, getGPL=FALSE)
    metadata_files[[dataset]] <- list()

    n_platforms <- length(gset)
    for(i in c(1:n_platforms)){
        metadata <- pData(gset[[i]]) %>% as.data.frame()
        write.table(metadata %>% as.data.frame() %>% rownames_to_column("Sample_geo_ID"), 
            file = paste0("before/", dataset, "/", i, "_metadata.tsv"), sep = "\t", row.names = F,quote = FALSE)

        metadata_files[[dataset]][[i]] <- paste0("before/", dataset, "/", i, "_metadata.tsv")
        if(is.null(samples_IDs)){
            samples_IDs <- metadata %>% rownames()
        }else{
            # add new samples, but without duplicates
            samples_IDs <- c(samples_IDs, metadata %>% rownames()) %>% unique()
        }
    }
}

Found 1 file(s)

GSE6008_series_matrix.txt.gz

Found 1 file(s)

GSE14407_series_matrix.txt.gz

Found 1 file(s)

GSE26712_series_matrix.txt.gz

Found 1 file(s)

GSE40595_series_matrix.txt.gz

Found 1 file(s)

GSE36668_series_matrix.txt.gz

Found 1 file(s)

GSE69428_series_matrix.txt.gz



## Read and preprocess metadata

In [5]:
# "GSE6008"
prepare_GSE6008 <- function(metadata){
    metadata <- metadata %>%
        # remove all columns with mutation in colname
        select(-c(grep("mutation", colnames(metadata), value = T))) %>%
        select(-c(grep("characteristics_ch1", colnames(metadata), value = T))) %>%
        select(-c("CEL_file_name:ch1", "P53_immunohistochemistry:ch1", "B-canenin_nuclear_accumulation:ch1", "PTEN_immunohistochemistry:ch1")) %>%
        select(-c("title")) %>%
        rename("tissue" = "source_name_ch1", "Grade" = "grade:ch1", "Stage" = "stage:ch1", "Tumor_Type" = "Tumor_Type:ch1") %>%
        mutate(
            Status = ifelse(grepl("Normal", tissue), "normal", "ovarian tumour"),
            Grade2 = ifelse(Grade == 1, "Low", ifelse(Grade == 2 | Grade == 3 | Grade == "2-3", "High", NA)),
            Stage = ifelse(Stage == "N/A", NA, toupper(gsub(" ", "", Stage))),
            Grade = ifelse(Grade == "N/A", NA, Grade),
            Tumor_Type = ifelse(Tumor_Type == "N/A", NA, Tumor_Type),
        ) %>%
        mutate(
            HistSubtypes = ifelse(grepl("Serous", Tumor_Type) & Grade2 == "High", "high-grade serous carcinoma",
                ifelse(grepl("Serous", Tumor_Type) & Grade2 == "Low", "low-grade serous carcinoma",
                        ifelse(grepl("Endometrioid", Tumor_Type), "endometrioid carcinoma",
                            ifelse(grepl("Clear_Cell", Tumor_Type), "clear cell carcinoma",
                                ifelse(grepl("Mucinous", Tumor_Type), "mucinous carcinoma", NA)
                            )
                        )
                    )
                 ),
            ClinicopathologicSubtypes = ifelse(HistSubtypes == "high-grade serous carcinoma", "Type II", "Type I"),
            Stage2 = ifelse(Stage %in% c("3A","3C","3B","3", "3D"), "III", 
                ifelse(Stage %in% c("1A","1C", "1A", "1"), "I", 
                    ifelse(Stage %in% c("4"), "IV", 
                        ifelse(Stage %in% c("2B", "2C", "2A", "2"), "II", NA)))),
            Stage3 = NA
        ) %>%
        mutate(HistSubtypes = ifelse(is.na(HistSubtypes), ifelse(Tumor_Type == "Serous", "serous carcinoma", NA), HistSubtypes)) %>%
        select(-c("Tumor_Type", "description"))
    metadata <- metadata %>%
        select(Sample_geo_ID, Status, HistSubtypes, Stage, Grade2, ClinicopathologicSubtypes, everything())
    return(metadata)
}


# "GSE26712"
prepare_GSE26712 <- function(metadata){
    
    metadata <- metadata %>%
        mutate(title = sub(" .*", "", title)) %>%
        rename("tissue" = "source_name_ch1", "Tissue" = "tissue:ch1") %>%
        select(-c("title", "characteristics_ch1", "characteristics_ch1.1", "characteristics_ch1.2", "characteristics_ch1.3",
            "surgery outcome:ch1", "survival years:ch1"))  %>%
        mutate(
            Stage = ifelse(grepl("Late-stage", Tissue), "Late(IIIB–IV)", NA),
            Grade2 = ifelse(grepl("high-grade", Tissue), "High", NA),
            SubtypeRep = ifelse(grepl("high-grade", Tissue), "Ser/PapSer", NA),
            Status = ifelse(grepl("Normal", Tissue), "normal", "ovarian tumour"),
            HistSubtypes = ifelse(SubtypeRep == "Ser/PapSer", "high-grade serous carcinoma", NA)) %>%
        mutate(
            ClinicopathologicSubtypes = ifelse(HistSubtypes == "high-grade serous carcinoma", "Type II", "Type I"),
            Stage2 = Stage,
            Stage3 = NA) %>%
        select(-c("treatment_protocol_ch1", "description", "Tissue"))
    metadata <- metadata %>% 
        select(Sample_geo_ID, Status, HistSubtypes, Stage, Grade2, ClinicopathologicSubtypes, everything())
        
    return(metadata)
}


# "GSE40595"

prepare_GSE40595 <- function(metadata){
    metadata <- metadata %>%
        filter(!grepl("stroma", title)) %>%
        mutate(
            Status = ifelse(grepl("Microdissected ovarian surface epthelium", source_name_ch1), "normal", "ovarian tumour"),
            tissue = "Ovary",
            HistSubtypes = ifelse(grepl("tumor", source_name_ch1), "high-grade serous carcinoma", NA),
            Grade2 = ifelse(grepl("tumor", source_name_ch1), "High", NA),
            ClinicopathologicSubtypes = ifelse(grepl("tumor", source_name_ch1), "Type II", NA),
            Stage = NA,
            Stage2 = NA,
            Stage3 = NA
        ) %>%
        select(-c("characteristics_ch1", "title", "tissue:ch1", "description", "source_name_ch1"))
    metadata <- metadata %>%
        select(Sample_geo_ID, Status, HistSubtypes, Stage, Grade2, ClinicopathologicSubtypes, everything())
    return(metadata)
}

# "GSE69428"
prepare_GSE69428 <- function(metadata){
    metadata <- metadata %>%
        filter(!grepl("FTSC", title)) %>%
        mutate(
            Grade2 = ifelse(grepl("HGSOC", title), "High", NA),
            ClinicopathologicSubtypes = ifelse(grepl("HGSOC", title), "Type II", NA),
            HistSubtypes = ifelse(grepl("HGSOC", title), "high-grade serous carcinoma", NA),
            Status = ifelse(grepl("normal", title), "normal", "ovarian tumour"),
            tissue = ifelse(grepl("HGSOC", title), "serous ovarian cancer", "oviduct"),
            Stage = NA,
            Stage2 = NA,
            Stage3 = NA
        ) %>%
        select(-c("treatment_protocol_ch1", "characteristics_ch1.1", "characteristics_ch1", 
            "sample type:ch1", "source_name_ch1", "description", "title", "tissue:ch1"))
    metadata <- metadata %>%
        select(Sample_geo_ID, Status, HistSubtypes, Stage, Grade2, ClinicopathologicSubtypes, everything())
    return(metadata)
}

# "GSE36668"
prepare_GSE36668 <- function(metadata){
    metadata <- metadata  %>%
        select(-c(grep("characteristics_ch1", colnames(metadata), value = T))) %>%
        filter(!grepl("borderline", title)) %>%
        rename("tissue" = "tissue:ch1") %>%
        mutate(
            Status = ifelse(grepl("normal", title), "normal", "ovarian tumour"),
            HistSubtypes = ifelse(grepl("carcinoma", title), "high-grade serous carcinoma", NA),
            Grade2 = ifelse(grepl("carcinoma", title), "High", NA),
            ClinicopathologicSubtypes = ifelse(grepl("carcinoma", title), "Type II", NA),
            Stage = ifelse(grepl("carcinoma", title), ifelse(grepl("1|2", title), "IIIC", "IV"), NA),
            Stage2 = ifelse(Stage %in% c("IIIA","IIIC","IIIB","III"), "III", 
                ifelse(Stage %in% c("IB","IC", "IA"), "I", 
                    ifelse(Stage %in% c("IV"), "IV", 
                        ifelse(Stage %in% c("IIB", "IIC", "IIA"), "II", NA)))),
            Stage3 = NA
        ) %>%
        select(-c("description", "treatment_protocol_ch1", "source_name_ch1")) %>%
        select(Sample_geo_ID, Status, HistSubtypes, Stage, Grade2, ClinicopathologicSubtypes, everything())
        
    return(metadata)
}

# "GSE14407"
prepare_GSE14407 <- function(metadata){
    metadata <- metadata %>%
        select(-c(grep("characteristics_ch1", colnames(metadata), value = T))) %>%
        select(-c("biomaterial_provider_ch1", "treatment_protocol_ch1")) %>%
        mutate(Stage = ifelse(grepl("CEPI", title), sub(".*_(.*)_[0-9]*", "\\1", title), NA)) %>%
        mutate(
            Stage = toupper(gsub(" ", "", Stage)),
            Status = ifelse(grepl("CEPI", title), "ovarian tumour", "normal"),
            HistSubtypes = ifelse(!grepl("OSE", title), "high-grade serous carcinoma", NA),
            Grade2 = ifelse(grepl("CEPI", title), "High", NA),
            ClinicopathologicSubtypes = ifelse(HistSubtypes == "high-grade serous carcinoma", "Type II", NA),
            Stage2 = ifelse(Stage %in% c("IIIA","IIIC","IIIB","III"), "III", 
                ifelse(Stage %in% c("IB","IC", "IA"), "I", 
                    ifelse(Stage %in% c("IV"), "IV", 
                        ifelse(Stage %in% c("IIB", "IIC", "IIA", "II"), "II", NA)))),
            Stage3 = NA
        ) %>%
        select(-c("title", "source_name_ch1", "description", "disease state:ch1", "specimen:ch1"))
    metadata <- metadata %>%
        select(Sample_geo_ID, Status, HistSubtypes, Stage, Grade2, ClinicopathologicSubtypes, everything())
    return(metadata)
}

In [6]:
metadata_list <- list()

for(dataset in c(
    "GSE6008", "GSE14407", "GSE26712",  "GSE40595", "GSE36668", "GSE69428"
)){
    cat("Dataset: ", dataset, "\n")
    for(i in c(1:length(metadata_files[[dataset]]))){
        cat("\t\t", i, "\n")
        metadata_file <- metadata_files[[dataset]][[i]]
        metadata <- read_tsv(metadata_file, show_col_types = FALSE)
        # check if metadata$type is RNA
        if("type" %in% colnames(metadata)){
            if(!("RNA" %in% metadata$type)){
                cat("Not RNA dataset\n")
        }}

        metadata <- metadata %>%
            select(-matches("^(geo_accession|status|submission_date|last_update_date|type|channel_count|organism_ch1)")) %>%
            select(-matches("^(molecule_ch1|taxid_ch1|data_row_count|relation|contact_)")) %>%
            select(-matches("^(extract_protocol_ch1|label_ch1|label_protocol_ch1|scan_protocol|hyb_protocol|data_processing)")) %>%
            select(-matches("^(growth_protocol_ch1|data_processing.1|patient id:ch1|age:ch1|supplementary_file.1|gender:ch1)"))
        
        if (dataset == "GSE6008") { metadata_list[[paste0(dataset, "_", i)]] <- prepare_GSE6008(metadata) }
        if (dataset == "GSE26712") { metadata_list[[paste0(dataset, "_", i)]] <- prepare_GSE26712(metadata) }
        if (dataset == "GSE40595") { metadata_list[[paste0(dataset, "_", i)]] <- prepare_GSE40595(metadata) }
        if (dataset == "GSE69428") { metadata_list[[paste0(dataset, "_", i)]] <- prepare_GSE69428(metadata) }
        if (dataset == "GSE36668") { metadata_list[[paste0(dataset, "_", i)]] <- prepare_GSE36668(metadata) }
        if (dataset == "GSE14407") { metadata_list[[paste0(dataset, "_", i)]] <- prepare_GSE14407(metadata) }
    }
}

Dataset:  GSE6008 
		 1 
Dataset:  GSE14407 
		 1 
Dataset:  GSE26712 
		 1 
Dataset:  GSE40595 
		 1 
Dataset:  GSE36668 
		 1 
Dataset:  GSE69428 
		 1 


In [7]:
# filter out samples that are not normal or HGSC ovarian cancer
for(dataset in names(metadata_list)){
    print(paste0("Dataset: ", dataset))
    print(paste0("Number of samples: ", nrow(metadata_list[[dataset]])))
    metadata_list[[dataset]] <- metadata_list[[dataset]] %>%
        filter(Status %in% c("normal", "ovarian tumour")) %>%
        filter(HistSubtypes %in% c("high-grade serous carcinoma", NA))

    print(paste0("Number of samples: ", nrow(metadata_list[[dataset]])))
}

[1] "Dataset: GSE6008_1"
[1] "Number of samples: 103"
[1] "Number of samples: 27"
[1] "Dataset: GSE14407_1"
[1] "Number of samples: 24"
[1] "Number of samples: 24"
[1] "Dataset: GSE26712_1"
[1] "Number of samples: 195"
[1] "Number of samples: 195"
[1] "Dataset: GSE40595_1"
[1] "Number of samples: 38"
[1] "Number of samples: 38"
[1] "Dataset: GSE36668_1"
[1] "Number of samples: 8"
[1] "Number of samples: 8"
[1] "Dataset: GSE69428_1"
[1] "Number of samples: 20"
[1] "Number of samples: 20"


## Save the results

In [8]:
library(readr)

for(dataset in names(metadata_list)){
    dataset_path <- gsub("_\\d$", "", dataset)
    # get i from dataset
    i <- gsub(".*_", "", dataset)
    write_tsv(metadata_list[[dataset]], paste0("before/", dataset_path, "/", i, "_metadata_short.tsv"))
}


In [9]:
# Function to process each dataset
process_dataset <- function(dataset) {
    # Read the metadata
    metadata <- metadata_list[[dataset]]
    # Get unique values for each column and format as a single string if multiple
    summarized <- metadata %>%
        summarise(across(everything(), ~paste(unique(.x), collapse = "; "))) %>%
        pivot_longer(cols = everything(), names_to = "column_name", values_to = "unique_values") %>%
        pivot_wider(names_from = column_name, values_from = unique_values)
    
    # Add dataset identifier
    summarized <- summarized %>% mutate(dataset = dataset)
    summarized$dataset <- as.character(dataset)
    # add column with total number of samples per dataset
    summarized$number_of_samples <- nrow(metadata)
    return(summarized)
}

# Process all datasets
processed_datasets <- map(names(metadata_list), process_dataset)

# Combine all processed datasets using full join, handling cases with no common columns
final_df <- reduce(processed_datasets, ~full_join(.x, .y, by = intersect(names(.x), names(.y)))) %>%
    as.data.frame()

# Print or view the final structured dataframe
# print(final_df)
final_df %>% write_tsv("preprocessing_info/metadata_summary.tsv", col_names = TRUE)

# Load the data

In [10]:
library(tidyverse)

library(affy)
library(GEOquery)


Attaching package: ‘affy’


The following object is masked from ‘package:lubridate’:

    pm




In [29]:
# Preparation (in case of errors)
# library(makecdfenv)

# make.cdf.package(
#     "GPL15048_HuRSTA_2a520709.CDF.gz",
#     packagename = 'hursta2a520709cdf',
#     # package.path = "./",
#     species = "Homo_sapiens",
#     cdf.path = "before/GSE66957_RAW/",
#     compress = TRUE)

# system("R CMD INSTALL hursta2a520709cdf", intern = TRUE)

In [11]:
options(timeout = max(300, getOption("timeout")))
options(download.file.method.GEOquery = "wget")

for(dataset in c(
    "GSE6008", "GSE14407", "GSE26712",  "GSE40595", 
    "GSE36668", "GSE69428"
    )){                   
    
    cat('Processing for dataset:', dataset)
    
    if (file.exists(paste0("before/", dataset, "/", dataset,"_RAW.tar"))) {
        untar(
            paste0("before/", dataset,"/", dataset, "_RAW.tar"), 
            exdir = paste0("before/", dataset, "_RAW")
        )
    } else {
        getGEOSuppFiles(dataset, baseDir="before")
        untar(
            paste0("before/", dataset,"/", dataset, "_RAW.tar"), 
            exdir = paste0("before/", dataset, "_RAW")
        )
    }

    # Save the metadata
    # gset <- getGEO(dataset, GSEMatrix=TRUE, getGPL=FALSE)
    gset = c(1)
    
    for(i in c(1:length(gset))){
        metadata <- read.table(paste0("before/", dataset, "/", i, "_metadata_short.tsv"), header = TRUE, sep = "\t")
        platform_files <- sub(".*/", "", metadata$supplementary_file)
        
        # if any file contains "txt.gz" ending, remove them from the list
        if(any(grepl("txt.gz", platform_files))){
            platform_files <- platform_files[!grepl("txt.gz", platform_files)]
            cat("Removed txt.gz files from the list. CHECK THE DATASET!")
        }
        
        platform_files_list <- paste0("before/", dataset, "_RAW/", platform_files)

        filenames <- as.character(unlist(platform_files_list))
        rawData <- ReadAffy(
            filenames = unlist(filenames),
            # celfile.path = paste0("before/", dataset, "_RAW"), 
            verbose = FALSE)

        # Prepare sample names
        sample_names <- sampleNames(rawData)
        sample_names <- sub("_.*", "", sample_names)
        sample_names <- sub(".CEL.gz$", "", sample_names)

        # Normalize the data
        normalizedData <- rma(rawData)
        expr_data <- exprs(normalizedData)
    
        # Save the normalized data
        rownames(expr_data) <- featureNames(normalizedData)
        colnames(expr_data) <- sample_names
        write.table(expr_data %>% as.data.frame() %>% rownames_to_column("row_ID"), 
            file = paste0("before/", dataset, "/", i, "_expr.tsv"), sep = "\t", row.names = F, quote = FALSE)
        
        print(paste0("Processed dataset: ", dataset, " ", i))
        print(paste0("Number of samples: ", ncol(expr_data)))
        print(paste0("Number of features: ", nrow(expr_data)))
    }
}

# remove all *_RAW folders from before
system("rm -r before/*_RAW")

Processing for dataset: GSE6008

Warning message:
“replacing previous import ‘AnnotationDbi::tail’ by ‘utils::tail’ when loading ‘hgu133acdf’”
Warning message:
“replacing previous import ‘AnnotationDbi::head’ by ‘utils::head’ when loading ‘hgu133acdf’”




Background correcting
Normalizing
Calculating Expression
[1] "Processed dataset: GSE6008 1"
[1] "Number of samples: 27"
[1] "Number of features: 22283"
Processing for dataset: GSE14407

Warning message:
“replacing previous import ‘AnnotationDbi::tail’ by ‘utils::tail’ when loading ‘hgu133plus2cdf’”
Warning message:
“replacing previous import ‘AnnotationDbi::head’ by ‘utils::head’ when loading ‘hgu133plus2cdf’”



Attaching package: ‘hgu133plus2cdf’


The following objects are masked from ‘package:hgu133acdf’:

    i2xy, xy2i




Background correcting
Normalizing
Calculating Expression
[1] "Processed dataset: GSE14407 1"
[1] "Number of samples: 24"
[1] "Number of features: 54675"
Processing for dataset: GSE26712Background correcting
Normalizing
Calculating Expression
[1] "Processed dataset: GSE26712 1"
[1] "Number of samples: 195"
[1] "Number of features: 22283"
Processing for dataset: GSE40595Background correcting
Normalizing
Calculating Expression
[1] "Processed dataset: GSE40595 1"
[1] "Number of samples: 38"
[1] "Number of features: 54675"
Processing for dataset: GSE36668Background correcting
Normalizing
Calculating Expression
[1] "Processed dataset: GSE36668 1"
[1] "Number of samples: 8"
[1] "Number of features: 54675"
Processing for dataset: GSE69428Background correcting
Normalizing
Calculating Expression
[1] "Processed dataset: GSE69428 1"
[1] "Number of samples: 20"
[1] "Number of features: 54675"


## Preprocess the data

In [12]:
library(tidyverse)
library(WGCNA)

Loading required package: dynamicTreeCut

Loading required package: fastcluster


Attaching package: ‘fastcluster’


The following object is masked from ‘package:stats’:

    hclust





Attaching package: ‘WGCNA’


The following object is masked from ‘package:stats’:

    cor




In [13]:
platform_ids_list <- c()

for(dataset in c(
    "GSE6008", "GSE14407", "GSE26712",  "GSE40595", "GSE36668", "GSE69428"
    )){
    if(dataset == "GSE131978" | dataset == "GSE68928"){ 
        subsets_len <- c(1, 2)
    } else {subsets_len <- c(1) }

    for(i in subsets_len){
        #read the expression data and metadata file
        metadata <- read.table(paste0("before/", dataset, "/", i, "_metadata_short.tsv"), header = TRUE, sep = "\t")
        platform_name <- metadata$platform_id %>% unique()
        platform_ids_list <- c(platform_ids_list, platform_name) %>% unique()
    }
}

In [14]:
platform_ids_list

[1] "GPL96"  "GPL570"

In [15]:
libraries_list <- list()

parse_triple <- function(x, delimiter = " /// ") {
    parts <- strsplit(x, delimiter)[[1]]
    # parts <- sort(parts)
    return(parts[1])
}


for (lib_name in platform_ids_list){
    cat("Processing", lib_name, "\n")
    lib_df <- read_tsv(paste0("preprocessing_info/libraries/", lib_name, ".txt"), comment="#", show_col_types = FALSE)
    
    if(lib_name %in% c("GPL570", "GPL96")){
        lib_df <- lib_df %>% 
                  select(ID, GB_ACC, `Gene Symbol`, ENTREZ_GENE_ID) %>%
                  rename(GeneSymbol = `Gene Symbol`) %>%
                  mutate(GeneSymbol = sapply(GeneSymbol, parse_triple, delimiter = " /// "),
                         ENTREZ_GENE_ID = sapply(ENTREZ_GENE_ID, parse_triple, delimiter = " /// "))
    }

    lib_df <- lib_df[!duplicated(lib_df$ID),]
    libraries_list[[lib_name]] <- lib_df
}

Processing GPL96 
Processing GPL570 


In [16]:
for(lib_name in names(libraries_list)){
    #print len of unqiue values of each column
    cat(lib_name, "\n")
    print(libraries_list[[lib_name]] %>% summarise(across(everything(), ~length(unique(.x)))))
}

GPL96 
# A tibble: 1 × 4
     ID GB_ACC GeneSymbol ENTREZ_GENE_ID
  <int>  <int>      <int>          <int>
1 22283  21129      13238          12938
GPL570 
# A tibble: 1 × 4
     ID GB_ACC GeneSymbol ENTREZ_GENE_ID
  <int>  <int>      <int>          <int>
1 54675  51277      22881          21181


In [17]:
length(intersect(unique(libraries_list[["GPL570"]]$GB_ACC), unique(libraries_list[["GPL96"]]$GB_ACC)))
length(intersect(unique(libraries_list[["GPL570"]]$GeneSymbol), unique(libraries_list[["GPL96"]]$GeneSymbol)))
length(intersect(unique(libraries_list[["GPL570"]]$ENTREZ_GENE_ID), unique(libraries_list[["GPL96"]]$ENTREZ_GENE_ID)))

[1] 21129

[1] 13238

[1] 12938

In [18]:
platfrom_list <- list()

for(dataset in c(
    "GSE6008", "GSE14407", "GSE26712",  "GSE40595", "GSE36668", "GSE69428"
    )){
    
    cat('\n\nProcessing for dataset:', dataset, "\n")
    if(dataset == "GSE131978" | dataset == "GSE68928"){ 
        subsets_len <- c(1, 2)
    } else {subsets_len <- c(1) }

    for(i in subsets_len){
        #read the expression data and metadata file
        metadata <- read.table(paste0("before/", dataset, "/", i, "_metadata_short.tsv"), header = TRUE, sep = "\t")
        expr_data <- read.table(paste0("before/", dataset, "/", i, "_expr.tsv"), header = TRUE, sep = "\t")
        cat('Loaded the data, shape of metadata:', dim(metadata), 'shape of expr_data:', dim(expr_data), '\n')

        platform_name <- metadata$platform_id %>% unique()

        # MaxVar# Collapse to GB_ACC
        gene_names <- libraries_list[[platform_name]]
        gene_names <- gene_names[!is.na(gene_names$GB_ACC),]
        expr_data_2 <- expr_data %>% filter(row_ID %in% gene_names$ID) %>% column_to_rownames("row_ID")
        cat('After filtering, shape of expr_data:', dim(expr_data_2), '\n')

        expr_data_ENTREZ <- WGCNA::collapseRows(
            expr_data_2, 
            rowGroup = gene_names$GeneSymbol, 
            rowID = gene_names$ID,
            method="maxRowVariance", connectivityBasedCollapsing=FALSE,
            methodFunction=NULL, connectivityPower=1,
            selectFewestMissing=TRUE, thresholdCombine=NA)$datETcollapsed
        cat('After collapsing rows (Entrez IDs), shape of expr_data:', dim(expr_data_ENTREZ), '\n')
        # save to file
        write.table(expr_data_ENTREZ %>% as.data.frame() %>% rownames_to_column("row_ID"), 
            file = paste0("before/", dataset, "/", i, "_expr_GBacc_MaxVar.tsv"), sep = "\t", row.names = F, quote = FALSE)

}}

In [ ]:
GB_ACC_id_lists <- list()

for(dataset in c(
    "GSE6008", "GSE14407", "GSE26712",  "GSE40595", "GSE36668", "GSE69428"
    )){                   
    
    cat('\nProcessing for dataset:', dataset, "\n")
    if(dataset == "GSE131978" | dataset == "GSE68928"){ 
        subsets_len <- c(1, 2)
    } else {subsets_len <- c(1) }

    for(i in subsets_len){
        # read data
        expr_data <- read.table(paste0("before/", dataset, "/", i, "_expr_GBacc_MaxVar.tsv"), header = TRUE, sep = "\t")
        # get the  names
        GB_ACC_id_lists[[paste0(dataset, "_", i)]] <- expr_data$row_ID
    }
} 


Processing for dataset: GSE6008 

Processing for dataset: GSE14407 

Processing for dataset: GSE26712 

Processing for dataset: GSE40595 

Processing for dataset: GSE36668 

Processing for dataset: GSE69428 


In [ ]:
library(jsonlite)

write_json(GB_ACC_id_lists, "/home/yuliya/repos/cosybio/FedComBat/datasets/Ovarian_Cancer_microarray/preprocessing_info/GBacc_IDs.json")


Attaching package: ‘jsonlite’


The following object is masked from ‘package:purrr’:

    flatten


